# Synthesize land model output from perturbed parameter ensemble

This script evaluates model output from a set of ensemble members in a perturbed parameter experiment. It identifies the best-performing ensemble members

## Import modules

In [111]:
import sys
#Path to the esm_tools.py script
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools/process_output')
import os
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools
import netCDF4 as nc4
import importlib
import esm_tools
import esm_viz
importlib.reload(esm_tools)
importlib.reload(esm_viz)
import math
pd.set_option('display.max_rows', 500)
import re

## User-defined parameters

In [124]:
# Just testing script?
test=False

# Make figures?
visualize = False

# How many years of data to average over for the structure variables
last_n_years=3

# For tree stem density
dbh_min = 10

# Calculate variables that require loading much more data (e.g. fire regime?)
decadal_scale_metrics=True

# How many years of data to average over for the fire variables
decadal_n_years=50

# File name of the file that stores the parameter ranges for the ensemble
param_range_file_name = 'param_ranges_100223.csv'

# Single case or multi case?
apply_to_one_case = False


##################
# If single case #
##################

if apply_to_one_case == True:

    case_name = 'STAN_equilibrium_110623_-17e2acb6a_FATES-8a054a12'

    # Subdirectory where the parameter file for each ensemble member is stored
    param_sub_dir="STAN_equilibrium_110623"
    
    print("Synthesizing output from this case:",case_name)

#################
# For multicase #
#################

if apply_to_one_case == False:

    case_name_prefix = 'CZ2_equilibrium_110323_'
    case_name_suffix = '_-17e2acb6a_FATES-8a054a12'
    param_sub_dir_prefix = "CZ2_equilibrium_110323_" # with underscore

    # Which cases in the group do you want to run?
    case_numbers = [1,2,3,4] # without leading zero
    case_tags = [str(case_num).rjust(2, '0') for case_num in case_numbers]
    case_names = [case_name_prefix + case_tag + case_name_suffix for case_tag in case_tags]
    print("Running from these cases:",case_names)

# Do you want to create a shortlist of the instances to process?
short_list_instances = True

inst_check = None
if short_list_instances == True:
    
    # If so, then provide lists of the case tag and the instance number (within lead zeroes) for each instance 
    def extract_four_digit_numbers(input_string):
        # Define a regular expression pattern to match a four-digit number after an underscore
        pattern = r'_\d{4}'

        # Use re.findall to find all matches of the pattern in the input string
        matches = re.findall(pattern, input_string)

        # Extract the four-digit numbers from the matches
        four_digit_numbers = [match.strip('_') for match in matches]

        return four_digit_numbers[-1]

    def extract_two_digit_numbers(input_string):
        # Define a regular expression pattern to match a four-digit number after an underscore
        pattern = r'_\d{2}_'

        # Use re.findall to find all matches of the pattern in the input string
        matches = re.findall(pattern, input_string)

        # Extract the four-digit numbers from the matches
        four_digit_numbers = [match.strip('_') for match in matches]

        return four_digit_numbers[-1]

    shortlist_df = pd.read_csv('/glade/scratch/adamhb/processed_output/CZ2_equilibrium_passing/CZ2-PEAS_passing_110923.csv')
    list(shortlist_df.columns)
    short_list_inst_ids = shortlist_df['inst_id']


    case_tags_short_list = []
    inst_short_list = []

    for i in short_list_inst_ids:
        if '110323' in i:
            print(i)
            case_tags_short_list.append(extract_two_digit_numbers(i))
            inst_short_list.append(extract_four_digit_numbers(i))
    
     
    inst_check = pd.DataFrame(inst_short_list)
    inst_check['case_tags'] = case_tags_short_list
    inst_check.columns = ['inst','case_tag']
    
    
# Optional
case_path = None
manual_case_path = None

Running from these cases: ['CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12', 'CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12', 'CZ2_equilibrium_110323_03_-17e2acb6a_FATES-8a054a12', 'CZ2_equilibrium_110323_04_-17e2acb6a_FATES-8a054a12']
CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0023
CZ2_equilibrium_110323_03_-17e2acb6a_FATES-8a054a12_0029
CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0002
CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0025
CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12_0033
CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0001
CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12_0011
CZ2_equilibrium_110323_04_-17e2acb6a_FATES-8a054a12_0004
CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12_0019
CZ2_equilibrium_110323_04_-17e2acb6a_FATES-8a054a12_0007
CZ2_equilibrium_110323_02_-17e2acb6a_FATES-8a054a12_0004
CZ2_equilibrium_110323_04_-17e2acb6a_FATES-8a054a12_0029
CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12_0006
CZ2_equilibr

## Define paths and script parameters

In [125]:
pft_names = ["pine","cedar","fir","shrub","oak"]

# Benchmarking metrics
my_metrics = ["BA","AGB","TreeStemD","TreeStemD_40","TreeStemD_60","TreeStemD_80","TreeStemD_100","ResproutD_oak","ResproutD_shrub","ShannonE","NPP","FailedPFTs",
              "Pct_shrub_cover_canopy","Pct_shrub_cover",
              "Combustible_fuel"]

if decadal_scale_metrics == True:
    my_metrics.extend(["Burned_area","Pct_high_severity_1700","Pct_high_severity_3500"])

# Path where case output lives
case_output_root = '/glade/scratch/adamhb/archive'

# Path to ensemble params
params_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles'

# Path to put any processed output
processed_output_root = '/glade/scratch/adamhb/processed_output'

# Path to param range files
param_range_root = '/glade/u/home/adamhb/california-fates/parameter_ranges/param_range_archive'

print("Calculating the following variables:",my_metrics)

Calculating the following variables: ['BA', 'AGB', 'TreeStemD', 'TreeStemD_40', 'TreeStemD_60', 'TreeStemD_80', 'TreeStemD_100', 'ResproutD_oak', 'ResproutD_shrub', 'ShannonE', 'NPP', 'FailedPFTs', 'Pct_shrub_cover_canopy', 'Pct_shrub_cover', 'Combustible_fuel', 'Burned_area', 'Pct_high_severity_1700', 'Pct_high_severity_3500']


## Variables to import

In [126]:
# Keep first two no matter what. They are needed to unravel multi-plexed dimensions
fields = ['FATES_SEED_PROD_USTORY_SZ','FATES_VEGC_AP','FATES_BURNFRAC',
          'FATES_NPLANT_PF','FATES_NPLANT_SZPF','FATES_NPLANT_RESPROUT_PF','FATES_FIRE_INTENSITY_BURNFRAC','FATES_IGNITIONS',
          'FATES_MORTALITY_FIRE_SZPF','FATES_BASALAREA_SZPF','FATES_CANOPYCROWNAREA_APPF','FATES_CANOPYCROWNAREA_PF','FATES_CROWNAREA_PF',
          'FATES_CROWNAREA_APPF','FATES_FUEL_AMOUNT_APFC','FATES_NPLANT_SZPF','FATES_FUEL_AMOUNT_APFC',
          'FATES_PATCHAREA_AP','FATES_CROWNAREA_PF','FATES_VEGC_ABOVEGROUND','FATES_NPP_PF']

## Benchmarking functions

In [127]:
def setup_benchmarking_data_structure(metrics,parameters,pft_names):
    
    metrics_out = metrics.copy()
    
    # add pft-specific vars
    pft_specific_ba_metrics = ["BA_" + pft for pft in pft_names]  
    metrics_out.extend(pft_specific_ba_metrics)
    
    # add inst tag
    metrics_out.append("inst")
    metrics_out.append("inst_id")
    metrics_out.append("param_file_path")
    metrics_out.extend(parameters)
    
    benchmarking_dict = {}
    for i in metrics_out:
        benchmarking_dict[i] = []
    return benchmarking_dict

def skip_inst(inst_check,case_tag,inst):
    qualifying_inst_tags = inst_check.loc[inst_check['case_tag'] == case_tag]['inst']
    if inst in list(qualifying_inst_tags):
        return False
    else:
        return True


def get_benchmarks(case_name,metrics,last_n_years,param_sub_dir,param_range_file_name,
                   test = False, pft_names = np.array(["pine","cedar","fir","shrub","oak"]),
                   pft_colors = ['gold','darkorange','darkolivegreen','brown','springgreen'],
                   param_range_root = param_range_root,
                   params_root = params_root,
                   manual_case_path = None, decadal_scale_metrics = False, decadal_n_years = 50,
                   inst_check = inst_check):
    
    
    print("Processing case:",case_name)
    
    # 1. Get info about the case
    if manual_case_path != None:
        full_case_path = manual_case_path
    
    else:
        full_case_path = esm_tools.get_path_to_sim(case_name,case_output_root)
    
    inst_tags = esm_tools.get_unique_inst_tags(full_case_path)
    
    if test == True:
        inst_tags = inst_tags[:3]
    
    n_inst = len(inst_tags)
    print("ninst:",n_inst)
    
    # 2. Set up the benchmarking data structure
    perturbed_params_df = pd.read_csv(os.path.join(param_range_root,param_range_file_name))
    perturbed_params = []
    for i in range(len(perturbed_params_df)):
        perturbed_params.append(perturbed_params_df['param'][i] + "_" + str(perturbed_params_df['pft'][i]))
    
    bench_dict = setup_benchmarking_data_structure(metrics,perturbed_params,pft_names)  
                                 
    # 3. Add param values to the data structure
    for inst in inst_tags:
        
        if short_list_instances == True:
            skip = skip_inst(inst_check,case_tag,inst)
            

            if skip == True:
                print("Skipping instance",inst)
                continue
        
        param_file_path = esm_tools.get_parameter_file_of_inst(params_root,param_sub_dir,inst)
        
        # Keep track of the parameter file path and the instance id for later use
        bench_dict['param_file_path'].append(param_file_path)
        bench_dict['inst_id'].append(case_name + "_" + inst)
        
        
        for i in perturbed_params_df.index:
            
            d = perturbed_params_df.loc[i]
            param = d['param']
            pft_index = max(0,int(d['pft'] - 1))
            organ = d['organ']
            
            if (param == "fates_frag_maxdecomp") & (organ > 1):
                continue
                
            if math.isnan(organ):
                organ_index = None
            else:
                organ_index = int(organ - 1)
           
            bench_dict[perturbed_params[i]].append(esm_tools.extract_variable_from_netcdf_specify_organ(
                                                           param_file_path,param,pft_index,organ_index))
    
    # 4. Add the model output to the data structure
    for inst in inst_tags:
        
        if short_list_instances == True:
            skip = skip_inst(inst_check,case_tag,inst)

            if skip == True:
                continue
        
        print("Working on ensemble member",inst,"of",len(inst_tags),"members")
        
        # Import the model output data for one ensemble member
        inst_files_last_n_years = esm_tools.get_files_of_inst(full_case_path,
                                                 inst,
                                                 last_n_years)
        
        ds = esm_tools.multiple_netcdf_to_xarray(inst_files_last_n_years,fields)
         
        bench_dict['inst'].append(inst)
        
        ## Basal area [m2 ha-1] ##
        if "BA" in bench_dict.keys():
            
            ## Pft-specific BA
            pft_level_ba = esm_tools.get_pft_level_basal_area(ds)
            
            for i in range(len(pft_names)):
                pft_name = pft_names[i]
                bench_dict['BA_' + pft_name].append(pft_level_ba[i])
            
            ## Shannon equitability index (wrt BA) ##
            bench_dict['ShannonE'].append(esm_tools.shannon_equitability(pft_level_ba))
            
            ## Number of failed pfts ##
            bench_dict['FailedPFTs'].append(esm_tools.get_n_failed_pfts(pft_level_ba,ba_thresh=0.1))
            
            ## Total BA
            pft_level_ba_no_shrub = np.delete(pft_level_ba,3)
            bench_dict['BA'].append(pft_level_ba_no_shrub.sum())
                  
        ## Stem density [N ha-1] ##
        if "TreeStemD" in bench_dict.keys():
            
            ## Total tree stem density >10 cm
            bench_dict["TreeStemD"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=dbh_min))
            bench_dict["TreeStemD_40"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=40))
            bench_dict["TreeStemD_60"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=60))
            bench_dict["TreeStemD_80"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=80))
            bench_dict["TreeStemD_100"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=100))
        
        if "ResproutD_oak" in bench_dict.keys():
            bench_dict["ResproutD_oak"].append(esm_tools.get_resprout_stem_den(ds,4))
            
        if "ResproutD_shrub" in bench_dict.keys():
            bench_dict["ResproutD_shrub"].append(esm_tools.get_resprout_stem_den(ds,3))
        
        ## AGB [kg C m-2]
        if "AGB" in bench_dict.keys():
            bench_dict["AGB"].append(esm_tools.get_AGB(ds))
        
        ## Total NPP [kg C m-2]
        if "NPP" in bench_dict.keys():
            bench_dict["NPP"].append(esm_tools.get_total_npp(ds))
        
        ## Shrub canopy layer cover [m2 m-2]
        if "Pct_shrub_cover_canopy" in bench_dict.keys():
            bench_dict["Pct_shrub_cover_canopy"].append(esm_tools.get_pft_level_crown_area(ds,pft_index = 3))
            
        if "Pct_shrub_cover" in bench_dict.keys():    
            bench_dict["Pct_shrub_cover"].append(esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = False))
        
        ## Fuel Load
        if "Combustible_fuel" in bench_dict.keys():
            bench_dict["Combustible_fuel"].append(esm_tools.get_combustible_fuel(ds))
        
        if decadal_scale_metrics == True:
            inst_files_decadal = esm_tools.get_files_of_inst(full_case_path,
                                                 inst,
                                                 decadal_n_years)
            ds_decadal = esm_tools.multiple_netcdf_to_xarray(inst_files_decadal,fields)
            
            if "Burned_area" in bench_dict.keys():
                bench_dict["Burned_area"].append(esm_tools.get_mean_annual_burn_frac(ds_decadal))
                
            if "Pct_high_severity_1700" in bench_dict.keys():
                bench_dict["Pct_high_severity_1700"].append(esm_tools.get_PHS_FLI_thresh(ds_decadal,1700))
            
            if "Pct_high_severity_3500" in bench_dict.keys():
                bench_dict["Pct_high_severity_3500"].append(esm_tools.get_PHS_FLI_thresh(ds_decadal,3500))
            
        
    return bench_dict

## Apply to case

In [116]:
if apply_to_one_case == True:
    output_dict = get_benchmarks(case_name=case_name,
                                  metrics = my_metrics,
                                  last_n_years=last_n_years,
                                  param_sub_dir=param_sub_dir,
                                  param_range_file_name = param_range_file_name,
                                  pft_names = pft_names,
                                  test=test,
                                  decadal_n_years=decadal_n_years,
                                  decadal_scale_metrics=decadal_scale_metrics)#,
                                  #manual_case_path = manual_case_path)
    df = pd.DataFrame(output_dict)
    if decadal_scale_metrics == True:
        file_name = case_name + "_with_fire_metrics"
    else:
        file_name = case_name + "_structure_metrics_last" + str(last_n_years) + "_years" 
    
    esm_tools.store_output_csv(case_name,file_name,df,processed_output_root = processed_output_root)

## View synthesized ensemble data

In [117]:
if apply_to_one_case == True:
    cols = list(df.columns)[:17]
    df.sort_values("FailedPFTs",ascending=True)[cols]

## Apply to multiple cases

In [128]:
if apply_to_one_case == False:
    df = pd.DataFrame()
    for case_tag in case_tags: 
        
        case_name = case_name_prefix + case_tag + case_name_suffix
        
        # Subdirectory where the parameter file for each ensemble member is stored
        param_sub_dir=param_sub_dir_prefix + case_tag
        output_dict = get_benchmarks(case_name=case_name,
                                  metrics = my_metrics,
                                  last_n_years=last_n_years,
                                  param_sub_dir=param_sub_dir,
                                  param_range_file_name = param_range_file_name,
                                  pft_names = pft_names,
                                  test=test,
                                  decadal_n_years=decadal_n_years,
                                  decadal_scale_metrics=decadal_scale_metrics)#,
                                  #manual_case_path = manual_case_path)
        
        
        
        df_tmp = pd.DataFrame(output_dict)
        df = pd.concat([df,df_tmp],axis = 0)
    
    time_stamp = esm_tools.print_current_datetime_no_spaces()
    processed_output_folder_name = case_name_prefix + "XX" + case_name_suffix + "_" + time_stamp
    
    if decadal_scale_metrics == True:
        file_name = processed_output_folder_name + "_with_fire_metrics"
    else:
        file_name = processed_output_folder_name + "_structure_metrics_last" + str(last_n_years) + "_years"
    
    time_stamp = esm_tools.print_current_datetime_no_spaces()
    processed_output_folder_name = case_name_prefix + "XX" + case_name_suffix + "_" + time_stamp
    esm_tools.store_output_csv(processed_output_folder_name,file_name,df,processed_output_root = processed_output_root)

Processing case: CZ2_equilibrium_110323_01_-17e2acb6a_FATES-8a054a12
ninst: 36
Skipping instance 0001
Skipping instance 0002
Skipping instance 0003
Skipping instance 0004
Skipping instance 0005
Skipping instance 0007
Skipping instance 0008
Skipping instance 0009
Skipping instance 0010
Skipping instance 0012
Skipping instance 0013
Skipping instance 0014
Skipping instance 0015
Skipping instance 0016
Skipping instance 0017
Skipping instance 0018
Skipping instance 0020
Skipping instance 0021
Skipping instance 0022
Skipping instance 0023
Skipping instance 0024
Skipping instance 0025
Skipping instance 0026
Skipping instance 0027
Skipping instance 0028
Skipping instance 0029
Skipping instance 0030
Skipping instance 0031
Skipping instance 0032
Skipping instance 0034
Skipping instance 0035
Skipping instance 0036
Working on ensemble member 0006 of 36 members


/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/ada

Working on ensemble member 0011 of 36 members


/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/ada

Working on ensemble member 0019 of 36 members


...

Working on ensemble member 0029 of 36 members


/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/ada

Working on ensemble member 0032 of 36 members


/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/adamhb/opt/miniconda/envs/env4/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/glade/work/ada

Directory '/glade/scratch/adamhb/processed_output/CZ2_equilibrium_110323_XX_-17e2acb6a_FATES-8a054a12_20231109162856' created successfully!


## Visualize

In [119]:
# if visualize == True:
#     col_selector = ["fates" in i for i in df.columns]
#     perturbed_params = df.columns[col_selector]


#     for p in perturbed_params:
#         esm_viz.plot_multi_panel(df = df, x_col = p,
#                                  y_cols = my_metrics, figsize=(12, 16),
#                                  save_fig=True,
#                                  output_path_for_case=output_path_for_case)

In [121]:
a = pd.read_csv('/glade/scratch/adamhb/processed_output/CZ2_equilibrium_110523_XX_-17e2acb6a_FATES-8a054a12_20231109154111/ensemble_output_CZ2_equilibrium_110523_XX_-17e2acb6a_FATES-8a054a12_20231109154111_with_fire_metrics.csv')

In [122]:
a[['Burned_area','Pct_high_severity_1700','Pct_high_severity_3500']]

,Burned_area,Pct_high_severity_1700,Pct_high_severity_3500
0,0.020,84.302326,67.441860
1,0.052,40.666667,11.333333
2,0.060,4.511278,0.000000
3,0.043,0.729927,0.000000
4,0.051,34.426230,5.737705
5,0.012,43.636364,15.151515
6,0.034,11.594203,0.000000
7,0.050,43.220339,10.169492
8,0.051,13.970588,0.000000
9,0.049,20.000000,1.428571
